<a href="https://colab.research.google.com/github/juhumkwon/Defense_Cloud/blob/main/9_0_2_Word_embedding_Positional_Encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import tensorflow as tf
import numpy as np

# 1. 문장 처리 (토큰화, 임베딩, 포지셔널 인코딩 추가)

# 예시 문장
tokens = ["나는", "학교에", "간다"]

# 어휘 크기와 임베딩 차원 정의
vocab_size = 10000  # 어휘 크기
embedding_dim = 512  # 임베딩 차원

# 예시 토큰 ID (가정)
token_ids = [42, 23, 77]
token_ids = tf.constant(token_ids)
print("token_ids:", token_ids)
print()

# 임베딩 레이어 생성
embedding_layer = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)

# 토큰 ID → 임베딩 벡터
# 여기서 embedded_tokens는 문장의 각 단어를 임베딩 벡터로 표현한 결과입니다.
embedded_tokens = embedding_layer(token_ids)  # shape: (3, 512)
print("embedded_tokens:", embedded_tokens)
print()

# 포지셔널 인코딩 생성 함수
def get_positional_encoding(seq_len, embedding_dim):
    positions = tf.range(seq_len, dtype=tf.float32)  # 문장 길이에 대한 위치
    i = tf.range(embedding_dim, dtype=tf.float32)  # 임베딩 차원
    angle_rates = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(embedding_dim, tf.float32))
    angles = positions[:, tf.newaxis] * angle_rates[tf.newaxis, :]
    return tf.concat([tf.sin(angles), tf.cos(angles)], axis=-1)[:, :embedding_dim]

# 포지셔널 인코딩 추가
seq_len = len(tokens)
positional_encoding = get_positional_encoding(seq_len, embedding_dim)
input_with_positional_encoding = embedded_tokens + positional_encoding

print("임베딩 벡터 (단어 의미 + 포지셔널 인코딩):")
print("input_with_positional_encoding:", input_with_positional_encoding.numpy())
print()

# 2. 자기-어텐션 (Self-Attention) 계산
# Query, Key, Value 생성
query = tf.keras.layers.Dense(embedding_dim)(input_with_positional_encoding)  # shape: (3, 512)
key = tf.keras.layers.Dense(embedding_dim)(input_with_positional_encoding)  # shape: (3, 512)
value = tf.keras.layers.Dense(embedding_dim)(input_with_positional_encoding)  # shape: (3, 512)

print("Query shape:", query.shape)
print("Key shape:", key.shape)
print("Value shape:", value.shape)

# 어텐션 스코어 계산 (Q * K^T)
attention_scores = tf.matmul(query, key, transpose_b=True)  # shape: (3, 3)
print("attention_scores:", attention_scores.shape)
print()

# 소프트맥스를 통해 가중치 계산
attention_weights = tf.nn.softmax(attention_scores, axis=-1)  # shape: (3, 3)
print("attention_weights:", attention_weights)
print()

# 어텐션 출력 (Attention Weight * Value)
attention_output = tf.matmul(attention_weights, value)  # shape: (3, 512)

# 3. 결과 출력

print("\n어텐션 후 출력 벡터:")
print(attention_output.numpy())


token_ids: tf.Tensor([42 23 77], shape=(3,), dtype=int32)

embedded_tokens: tf.Tensor(
[[ 0.01057028  0.01876489  0.04978373 ...  0.01247923 -0.03887016
   0.03383276]
 [-0.01171038 -0.04157472 -0.03382719 ...  0.00186025 -0.00987711
   0.0023483 ]
 [ 0.0409399   0.04031232 -0.00321622 ...  0.00821297 -0.01329632
   0.04031802]], shape=(3, 512), dtype=float32)

임베딩 벡터 (단어 의미 + 포지셔널 인코딩):
input_with_positional_encoding: [[ 0.01057028  0.01876489  0.04978373 ...  0.01247923 -0.03887016
   0.03383276]
 [ 0.82976055  0.79989624  0.7880291  ...  0.00196771 -0.00977344
   0.00245197]
 [ 0.95023733  0.9496097   0.9331985  ...  0.00842789 -0.01308899
   0.04052535]]

Query shape: (3, 512)
Key shape: (3, 512)
Value shape: (3, 512)
attention_scores: (3, 3)

attention_weights: tf.Tensor(
[[0.31941542 0.33810255 0.3424821 ]
 [0.46646982 0.35079578 0.18273439]
 [0.44185343 0.38997218 0.16817433]], shape=(3, 3), dtype=float32)


어텐션 후 출력 벡터:
[[-0.01899781  0.07093623 -0.4377731  ...  0.05189446  0.2